# Experimentos: Baseline com QSVM

Este notebook estabelece baselines de performance com QSVM no dataset Student Performance

Usando Seleção de features com base 


In [ ]:
# Importando todas as ferramentas necessárias
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
import time

# Importando as ferramentas quânticas
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

# Dataset = Student Performance

In [ ]:
# Importando o dataset
student_performance = 'student-por.csv'

df_student_performance = pd.read_csv(student_performance, sep=',')  # Lendo o dataset com separador ','

# 3. Replicamos a separação que a biblioteca fazia para nós:
# y são as colunas de notas
y = df_student_performance[['G1', 'G2', 'G3']]
# X são TODAS as outras colunas
X = df_student_performance.drop(columns=['G1', 'G2', 'G3'])

# --- Verificações para confirmar que deu tudo certo ---
print("--- Dataset carregado localmente com sucesso! ---")
print("Formato das features (X):", X.shape)
print("Formato dos alvos (y):", y.shape)

print("\n5 primeiras linhas das features (X):")


'# Importando o dataset\nstudent_performance = \'student-por.csv\'\n\ndf_student_performance = pd.read_csv(student_performance, sep=\',\')  # Lendo o dataset com separador \',\'\n\n# 3. Replicamos a separação que a biblioteca fazia para nós:\n# y são as colunas de notas\ny = df_student_performance[[\'G1\', \'G2\', \'G3\']]\n# X são TODAS as outras colunas\nX = df_student_performance.drop(columns=[\'G1\', \'G2\', \'G3\'])\n\n# --- Verificações para confirmar que deu tudo certo ---\nprint("--- Dataset carregado localmente com sucesso! ---")\nprint("Formato das features (X):", X.shape)\nprint("Formato dos alvos (y):", y.shape)\n\nprint("\n5 primeiras linhas das features (X):")\n'

In [ ]:
X.head(5) 

In [ ]:
# Dimensões dos dados do Student Performance Dataset
print("\nDimensões do Student Performance Dataset:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")


'# Dimensões dos dados do Student Performance Dataset\nprint("\nDimensões do Student Performance Dataset:")\nprint(f"X: {X.shape}")\nprint(f"y: {y.shape}")'

In [ ]:
# Data cleaning
# check for missing values
print("\nVerificando valores ausentes:")
print(X.isnull().sum())

'# Data cleaning\n# check for missing values\nprint("\nVerificando valores ausentes:")\nprint(X.isnull().sum())'

## Pré-processamento dataset Student Performance

In [ ]:
# creating classification targets
y_final = np.where(y['G3'] >= 10, 1, 0)  # Convertendo a nota final em uma classificação binária (aprovado/reprovado). Usando 10 como o limite de aprovação.
# 1 representa aprovado e 0 representa reprovado.
# Verificando a distribuição das classes
print("\nDistribuição das classes (Aprovado/Reprovado):")
print(pd.Series(y_final).value_counts()) # Contando as ocorrências de cada classe. usando pd.Series para criar uma série pandas a partir do array numpy




'# creating classification targets\ny_final = np.where(y[\'G3\'] >= 10, 1, 0)  # Convertendo a nota final em uma classificação binária (aprovado/reprovado). Usando 10 como o limite de aprovação.\n# 1 representa aprovado e 0 representa reprovado.\n# Verificando a distribuição das classes\nprint("\nDistribuição das classes (Aprovado/Reprovado):")\nprint(pd.Series(y_final).value_counts()) # Contando as ocorrências de cada classe. usando pd.Series para criar uma série pandas a partir do array numpy'

In [ ]:
# The dataset seem unbalanced, so it has to be balanced

# tranforming text features into numerical features
X_final = pd.get_dummies(X, drop_first=True)  # Convertendo variáveis categóricas em variáveis dummy # drop_first=True evita a armadilha da variável fictícia
# Verificando as dimensões dos dados após a codificação
print("\nDimensões dos dados após a codificação:") 
print("Formato das features (X) antes do processamento:", X.shape)
print("Formato das features (X) após o processamento:", X_final.shape)
# Verificando as primeiras linhas dos dados codificados
print("\nPrimeiras linhas dos dados codificados:")
print(X_final.head())
print(X_final.dtypes)



'# The dataset seem unbalanced, so it has to be balanced\n\n# tranforming text features into numerical features\nX_final = pd.get_dummies(X, drop_first=True)  # Convertendo variáveis categóricas em variáveis dummy # drop_first=True evita a armadilha da variável fictícia\n# Verificando as dimensões dos dados após a codificação\nprint("\nDimensões dos dados após a codificação:") \nprint("Formato das features (X) antes do processamento:", X.shape)\nprint("Formato das features (X) após o processamento:", X_final.shape)\n# Verificando as primeiras linhas dos dados codificados\nprint("\nPrimeiras linhas dos dados codificados:")\nprint(X_final.head())\nprint(X_final.dtypes)'

## Spliting data into training and testing sets

In [ ]:
X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.25, random_state=42, stratify=y_final
)



'X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42, stratify=y_final)  # Stratify -> mantém a proporção das classes no split\n\nprint(f"\nDimensões do conjunto de treino: {X_train.shape}")\nprint(f"Dimensões do conjunto de teste: {X_test.shape}")'

## Applying feature selection by correlation (30 -> 5 features)

In [ ]:
train_df = X_train_full.copy()
train_df["aprovado"] = y_train

# Calculamos a correlação das features com a variável alvo
correlation = train_df.corr(numeric_only=True)["aprovado"].abs().sort_values(ascending=False)
# Selecionando as 5 features mais importantes (excluindo a própria 'aprovado')
N_FEATURES = 5
top_features = correlation[1:N_FEATURES+1].index.tolist()
print(f"\nTop {N_FEATURES} features selecionadas com base no treino:", top_features)

# 1c. Filtramos os conjuntos de treino e teste para conter APENAS essas features
X_train_selected = X_train_full[top_features]
X_test_selected = X_test_full[top_features]

## Normalizing 

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)
print("\nDados selecionados e normalizados.")

'scaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)  \nX_test_scaled = scaler.transform(X_test)  '

## Using SMOTE to balance

In [ ]:
# Using SMOTE to balance the dataset
from imblearn.over_sampling import SMOTE

print("Distribuição de classes no treino ANTES do SMOTE:")
print(pd.Series(y_train).value_counts())

# aplying SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Verificando o balanceamento DEPOIS do SMOTE
print("\nDistribuição de classes no treino DEPOIS do SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

'# Using SMOTE to balance the dataset\nfrom imblearn.over_sampling import SMOTE\n\nprint("Distribuição de classes no treino ANTES do SMOTE:")\nprint(pd.Series(y_train).value_counts())\n\n# aplying SMOTE\nsmote = SMOTE(random_state=42)\nX_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)\n\n# Verificando o balanceamento DEPOIS do SMOTE\nprint("\nDistribuição de classes no treino DEPOIS do SMOTE:")\nprint(pd.Series(y_train_resampled).value_counts())'

## Training the dataset

In [ ]:
# --- PASSO 4: Treinamento e Avaliação do QSVM ---
num_features = X_train_resampled.shape[1]
feature_map_sp = ZZFeatureMap(feature_dimension=num_features, reps=1)
fidelity_kernel_sp = FidelityQuantumKernel(feature_map=feature_map_sp)
qsvc_sp = QSVC(quantum_kernel=fidelity_kernel_sp, random_state=42)

print(f"\nModelo QSVC criado para {num_features} features. Iniciando treinamento...")

start_time = time.time()
qsvc_sp.fit(X_train_resampled, y_train_resampled)
end_time = time.time()
print(f"Modelo treinado em {end_time - start_time:.2f} segundos.")

qsvc_predictions_sp = qsvc_sp.predict(X_test_scaled)

print("\n--- Relatório de Classificação para o QSVM com Seleção de Features (Student Performance) ---")
print(classification_report(y_test, qsvc_predictions_sp, target_names=['Reprovado', 'Aprovado']))